<a href="https://www.kaggle.com/code/nurielreuven/blackjacks-hands-win-prediction-86-percision?scriptVersionId=95862007" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

By Nuriel Reuven

Version 1.0 , Uploaded on 10.5.22  |  2022- ©All Rights Reserved

# Background

![](https://upload.wikimedia.org/wikipedia/commons/4/4b/Blackjack_board.JPG)

Blackjack is a known casino game. The game is played between a player (or several players) and a dealer. The goal of the game is to have the highest sum of cards,but not exceeding 21. If the player's sum of cards is higher than the dealer's (and not exceeding 21), the player wins, And vice versa. Once the sum of the cards exceeds 21, the cards holder is busted.

Statistically the odds are not in the player's favor, since he is the first to play. This makes it a popular casino game. Several methods to tilt the odds to the player's favor have been developed (and proved mathematically), however in this notebook we will not use them and try to analyze a set of hands, extract statistics and try to develope a model to predict wether the player can win/lose/push (tie) in a round.

You do not need to know more than the basic rules of the game to understand this notebook. If you'd still wish to read more about the rules of the game, click [here](https://en.wikipedia.org/wiki/Blackjack#Rules_of_play_at_casinos) or [here ](https://www.youtube.com/watch?v=qd5oc9hLrXg&t=48s&ab_channel=Howcast)


The dataset contains simulation of 50M hands of blackjack with some exceptions to the rules. To read about the dataset,generator and exceptions click [here ](https://www.kaggle.com/datasets/dennisho/blackjack-hands)

# Data Exploration

In [ ]:
import pandas as pd
import numpy as np
import plotly_express as px
import ipywidgets as widgets
import ast
from math import nan
import warnings
warnings.filterwarnings('ignore')

We will use only 250000 samples as 50M requires way too much computational resources :)

In [ ]:
data = pd.read_csv('../input/blackjack-hands/blackjack_simulator.csv', sep = ',' , nrows = 250000)  # Because 5M is too much :)
data.head(10)

**Reminder:** S = Stand , H = Hit , D = Double , P = Split , R = Surrender , I = Insurance , N = No Insurance

*shoe_id* is not relevant for neither the analysis or model building, so we will remove it.*run_count* and *true_count* are used for [cards counting](https://en.wikipedia.org/wiki/Card_counting). We will not discuss it in the analysis, although it will be used for model building.

We can already see that when a player is splitting cards it does not mention the split in all hands in action taken (i.e index 6, the 2nd hand actions are  ['H', 'S'] and not  ['P','H','S']). We will correct that later.

In [ ]:
data.info()

No Null objects found!

In [ ]:
type(data['initial_hand'][1]) , type(data['dealer_final'][1])

We can see that the lists in our dataset are as string representation of list and not actual list, so we will convert all those columns to lists

In [ ]:
data['initial_hand'] = data['initial_hand'].apply(lambda x: ast.literal_eval(x))
data['dealer_final'] = data['dealer_final'].apply(lambda x: ast.literal_eval(x))
data['player_final'] = data['player_final'].apply(lambda x: ast.literal_eval(x))
data['player_final_value'] = data['player_final_value'].apply(lambda x: ast.literal_eval(x))	
data['actions_taken'] = data['actions_taken'].apply(lambda x: ast.literal_eval(x))	

Now after converting to list we are able to correct the split bug that was mentioned before, we will use for loop tho its not convinient

In [ ]:
data['actions_taken'] = data['actions_taken'].apply(lambda x: [x[0] , ['P'] + x[1]]  if len(x)==2 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2]]  if len(x)==3 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3]]  if len(x)==4 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3] , ['P'] + x[4]]  if len(x)==5 else x)

In [ ]:
data.iloc[6]

The bug was fixed

In [ ]:
data.info()

As we saw before,some lines have several hands info in 1 line (due to split, for example , like line 6 from before). We would like to do is to split to one hand per row. This would be easy using pandas .explode()

In [ ]:
data = data.explode(['player_final','player_final_value','actions_taken'])

In [ ]:
data.info()

7004 lines were added.
We would like to split the lists of player's and dealer's cards and players actions for future calculations. We will create a function to split them and apply it

In [ ]:
def cards_splitter(data,cards,text):
    stopsign = cards.apply(lambda x: len(x))
    for i in range(1,max(stopsign)):
        data[f'{text}{i}'] = cards.apply(lambda x: nan if len(x)<i else x[i-1])


In [ ]:
cards_splitter(data,data['player_final'],'player_card_')
cards_splitter(data,data['dealer_final'],'dealer_card_')
cards_splitter(data,data['actions_taken'],'action_taken_')

In [ ]:
data.sample(5)

In [ ]:
data.info()

Another thing we'd like to do is to replace 'BJ' to 21 to stay in int types only. To differ wether the hand was blackjack or just a sum of 21 we will add 2 new columns of 'is_blackjack_player' and 'is_blackjack_dealer'. 

In [ ]:
data['is_blackjack_dealer'] = data['dealer_final_value'].where(data['dealer_final_value'] == 'BJ').fillna(0).replace('BJ',1)
data['is_blackjack_player'] = data['player_final_value'].where(data['player_final_value'] == 'BJ').fillna(0).replace('BJ',1)

In [ ]:
data['dealer_final_value'] = data['dealer_final_value'].replace('BJ',21)
data['player_final_value'] = data['player_final_value'].replace('BJ',21)

In [ ]:
type(data['player_final_value'][0]) , type(data['dealer_final_value'][0])

We can see the player_final_value and dealer_final_value are represented as string instead of int, we will correct that:

In [ ]:
data['dealer_final_value'] = data['dealer_final_value'].astype('int64')
data['player_final_value'] = data['player_final_value'].astype('int64')

In [ ]:
type(data['player_final_value'][0]) , type(data['dealer_final_value'][0])

Last thing we need to do is to correct the win amount as it is normalized to the cases where each row contained several hands. It would be easy as we have all data about the cards and the rules of the game. We will not use the win amount but mark a pure Win/Loss/Push case.

In [ ]:
data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
data.loc[data['player_final_value'] > 21 , 'win'] = -1
data['win'] = data['win'].astype('int64')

 Win = 1 , Push = 0 , Loss = -1

To check our classification was correct:

In [ ]:
data['win']

In [ ]:
data.sample(5)

# Analysis

We will now analyse our dataset to get some statistics about the game.

In [ ]:
data['win'].value_counts()

In [ ]:
data['win'].value_counts(normalize=True)

We might think that our classification from before is incorrect, as from the distribution it looks like there is class bias. However, by looking for theoretical black jack probabilities, we can see that they are very close to the values we got, so our classification is correct!

More about blackjack probabilities: [here](https://www.onlinegambling.com/blackjack/odds/#:~:text=23%25-,Odds%20of%20Winning%20Blackjack,of%20a%20loss%20at%2049.10%25.)

In [ ]:
sum(data['win'])

The odds arent really for player's favor...

In [ ]:
data['dealer_up'].value_counts()

As expected 10 appears almost 4x more than other cards , since 10 is represented by 4 cards - 10,J,Q,K. Excluding that the cards are balanced.

In [ ]:
sum(data['is_blackjack_dealer']) / len(data['dealer_final_value']) , sum(data['is_blackjack_player']) / len(data['player_final_value'])

We can see approximately 4.6% of the hands ended in blackjack for either player or dealer, [corresponding to theorey](https://betandbeat.com/blackjack/blog/odds-of-getting-blackjack/#:~:text=It%20is%20simpler%20to%20calculate,the%20player%20are%20about%204.8%25.).

In [ ]:
data['actions_taken'].astype(str).value_counts().head(20)

The empty brackets [] seem suspicious. There are no null objects so one possible option is blackjack at first, as if that is the case the player has no action to do and the round is automatically over. We'll make sure that is the case.

In [ ]:
data[['dealer_up','win']].groupby('dealer_up')['win'].mean().sort_values(ascending=False)

In [ ]:
data['initial_hand'] = data['initial_hand'].astype(str)
data[['initial_hand','win']].groupby('initial_hand')['win'].mean().sort_values(ascending=False)

In [ ]:
data['initial_hand'] = data['initial_hand'].astype(str)
data[['initial_hand','win']].groupby('initial_hand')['win'].value_counts().head(25)

# Graphs

Now for a few graphs for the analysis.

We first start with a function to set 11 to 1 for hands with sum above 21, this will be helpful later:

In [ ]:
def hands_normalizer(data,maxhandsnum):
        for i in range(1,maxhandsnum+1):
            if i == 1:
                data.loc[(data.loc[:,'cards_sum'] > 21) & ((data.loc[:,f'player_card_{i}'] == 11) | (data.loc[:,f'player_card_{i+1}'] == 11)) ,'cards_sum']  -= 10
                continue
            elif i == 2:
                continue
            else:
                data.loc[(data.loc[:,'cards_sum'] > 21) & (data.loc[:,f'player_card_{i}'] == 11) ,'cards_sum']  -= 10

We first want to check what step should we take at a certain point. We will map all steps for each sum of cards we have and each dealer card that gives us the biggest probability of winning.

In [ ]:
def step_graph_generator(data,step):
    columns = ['dealer_up','win']
    hands = []
    for i in range(1,step+2):
        hands.append(f'player_card_{i}')
    columns = columns + [f'action_taken_{step}'] + hands
    graphdata = data[columns]
    graphdata['cards_sum'] = graphdata['player_card_1'] 
    for i in range(2,step+2):
        graphdata.loc[:,'cards_sum'] += graphdata.loc[:,f'player_card_{i}'] 
    hands_normalizer(graphdata,step+1)
    Graph = graphdata.query('win == 1')
    Graph['Counts'] = Graph.groupby(['cards_sum','dealer_up'])[f'action_taken_{step}'].transform('count')
    MaxIndex = Graph.groupby(['cards_sum','dealer_up'])['Counts'].idxmax().drop_duplicates()
    FinalGraph = Graph.loc[MaxIndex,:].sort_values(['cards_sum','dealer_up']).fillna('No Data')
    FinalGraph = FinalGraph.replace({ 'S' : 'Stand', 'H' : 'Hit' , 'D' : 'Double' , 'P' : 'Split' , 'R' : 'Surrender' , 'N' : 'No Insurance'})
    return px.scatter(FinalGraph, x='cards_sum', y='dealer_up', color=f'action_taken_{step}' , hover_data=[f'action_taken_{step}'] , labels={
                     'cards_sum': 'Player\'s Cards Sum', 'dealer_up': 'Dealer\'s Card Up', f'action_taken_{step}': 'Action'} ,
                      color_discrete_map={ 'Stand': '#FC2020', 'Hit': '#09D138' , 'Double': "#009999", 'Split': '#FF3399', 'Surrender': "#CCCC00", 'No Insurance': "#B266FF", 'No Data': '#C0C0C0'}, 
                      title=f'Step {step}').update_traces(marker=dict(size=15,line=dict(width=1,color='DarkSlateGrey')))

@widgets.interact
def show_action_graphs(step=widgets.IntSlider(value=1, min=1, max=6, step=1)):
    return step_graph_generator(data,step)

Now we will also map all actions not as most recommended but as histogram:

In [ ]:
def step_bar_graph_generator(data,step):
    columns = ['dealer_up','win']
    hands = []
    for i in range(1,step+2):
        hands.append(f'player_card_{i}')
    columns = columns + [f'action_taken_{step}'] + hands
    graphdata = data[columns]
    graphdata['cards_sum'] = graphdata['player_card_1'] 
    for i in range(2,step+2):
        graphdata.loc[:,'cards_sum'] += graphdata.loc[:,f'player_card_{i}'] 
    hands_normalizer(graphdata,step+1)
    Graph = graphdata.query('win == 1')
    Graph['Counts'] = Graph.groupby(['cards_sum','dealer_up'])[f'action_taken_{step}'].transform('count')
    MaxIndex = Graph.groupby(['cards_sum','dealer_up'])['Counts'].idxmax().drop_duplicates()
    FinalGraph = Graph.loc[MaxIndex,:].sort_values(['cards_sum','dealer_up']).fillna('No Data')
    FinalGraph = FinalGraph.replace({ 'S' : 'Stand', 'H' : 'Hit' , 'D' : 'Double' , 'P' : 'Split' , 'R' : 'Surrender' , 'N' : 'No Insurance'})
    return px.histogram(FinalGraph, x='cards_sum', color=f'action_taken_{step}' , hover_data=[f'action_taken_{step}'] , labels={
                     'cards_sum': 'Player\'s Cards Sum', f'action_taken_{step}': 'Action'} ,
                      color_discrete_map={ 'Stand': '#FC2020', 'Hit': '#09D138' , 'Double': "#009999", 'Split': '#FF3399', 'Surrender': "#CCCC00", 'No Insurance': "#B266FF", 'No Data': '#C0C0C0'}, 
                      title=f'Step {step}')

@widgets.interact
def show_bar_graphs(step=widgets.IntSlider(value=1, min=1, max=6, step=1)):
    return step_bar_graph_generator(data,step)

Recommended action per first 2 cards and dealer's up card:

In [ ]:
StepBy1stHand = data[['initial_hand','dealer_up','win','action_taken_1']]
StepBy1stHand['initial_hand'] = StepBy1stHand['initial_hand'].astype(str)
GraphD = StepBy1stHand.query('win == 1')
GraphD['Counts'] = GraphD.groupby(['initial_hand','dealer_up'])['action_taken_1'].transform('count')
MaxIndexD = GraphD.groupby(['initial_hand','dealer_up'])['Counts'].idxmax().drop_duplicates()
NewGraphD = GraphD.loc[MaxIndexD,:].sort_values(['initial_hand','dealer_up']).fillna('No Data')
NewGraphD = NewGraphD.replace({ 'S' : 'Stand', 'H' : 'Hit' , 'D' : 'Double' , 'P' : 'Split' , 'R' : 'Surrender' , 'N' : 'No Insurance'})
px.scatter(NewGraphD, x='initial_hand', y='dealer_up', color='action_taken_1', hover_data=['action_taken_1'] , labels={
                     'initial_hand': 'Player\'s Hand', 'dealer_up': 'Dealer\'s Card Up', 'action_taken_1': 'Action'} ,
                      color_discrete_map={ 'Stand': '#FC2020', 'Hit': '#09D138' , 'Double': "#009999", 'Split': '#FF3399', 'Surrender': "#CCCC00", 'No Insurance': "#B266FF", 'No Data': '#C0C0C0'}, 
                      title='Step 1 by first cards').update_traces(marker=dict(size=10,line=dict(width=1,color='DarkSlateGrey')))


And the matching histogram:

In [ ]:
StepBy1stHand = data[['initial_hand','dealer_up','win','action_taken_1']]
StepBy1stHand['initial_hand'] = StepBy1stHand['initial_hand'].astype(str)
GraphD = StepBy1stHand.query('win == 1')
GraphD['Counts'] = GraphD.groupby(['initial_hand','dealer_up'])['action_taken_1'].transform('count')
MaxIndexD = GraphD.groupby(['initial_hand','dealer_up'])['Counts'].idxmax().drop_duplicates()
NewGraphD = GraphD.loc[MaxIndexD,:].sort_values(['initial_hand','dealer_up']).fillna('No Data')
NewGraphD = NewGraphD.replace({ 'S' : 'Stand', 'H' : 'Hit' , 'D' : 'Double' , 'P' : 'Split' , 'R' : 'Surrender' , 'N' : 'No Insurance'})
px.histogram(NewGraphD, x='initial_hand', color='action_taken_1', hover_data=['action_taken_1'] , labels={
                     'initial_hand': 'Player\'s Hand', 'dealer_up': 'Dealer\'s Card Up', 'action_taken_1': 'Action'} ,
                      color_discrete_map={ 'Stand': '#FC2020', 'Hit': '#09D138' , 'Double': "#009999", 'Split': '#FF3399', 'Surrender': "#CCCC00", 'No Insurance': "#B266FF", 'No Data': '#C0C0C0'}, 
                      title='Step 1 by first cards')

Now we will try mapping the round results per the actions taken and sum of cards of the player/dealer:

In [ ]:
def win_by_action_graph_generator(data,step ,norm):
    graphdata = data[[f'action_taken_{step}','win']]
    graphdata = graphdata.replace({ 'S' : 'Stand', 'H' : 'Hit' , 'D' : 'Double' , 'P' : 'Split' , 'R' : 'Surrender' , 'N' : 'No Insurance' ,  1 : 'Win', 0 : 'Push' ,-1 : 'Loss'})
    if norm == False:
        return px.histogram(graphdata, x=f'action_taken_{step}', color='win' , hover_data=['win'] , text_auto=True , labels={
                      f'action_taken_{step}': 'Action'} , color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"},  
                      title=f'Step {step}')
    elif norm == True:
        return px.histogram(graphdata, x=f'action_taken_{step}', color='win' , hover_data=['win'] , text_auto=True , barnorm = 'percent' , labels={
                      f'action_taken_{step}': 'Action'} , color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"},  
                      title=f'Step {step}')

@widgets.interact
def show_win_by_action_graphs(step=widgets.IntSlider(value=1, min=1, max=6, step=1) , norm = widgets.ToggleButtons( options=[True,False] , value=False , description='Normalized')):
    return win_by_action_graph_generator(data,step , norm)

Whats interesting is that we can see that almost sweepingly standing gives us the best win chances, regardless of the hand!

We will create another graph to try to support this claim, we will plot the mean win as player's amount of cards:


In [ ]:
WinMean = pd.DataFrame(data['win'])
WinMean['len'] = data['player_final'].apply(lambda x: len(x))
WinMeanPlotData = pd.DataFrame(WinMean.groupby(['len'])['win'].mean())
px.scatter(WinMeanPlotData , x=WinMeanPlotData.index , y='win' , hover_data=['win'] , labels={
                     'len': 'Amount of player\'s cards holding', 'win' : 'Mean Win'}).update_traces(marker=dict(size=12,line=dict(width=1,color='DarkSlateGrey')))

This also supports our claim. The graph is monotonically decreasing.

Notice that when holding only 2 cards (meaning standing after getting the first 2 cards and immediately standing) the mean is **positive**!

Now we'll plot win distribution per sum of cards and current step:

In [ ]:
def win_graph_generator(data,step):
    hands = []
    for i in range(1,step+2):
        hands.append(f'player_card_{i}')
    columns = hands + ['win']
    graphdata = data[columns]
    graphdata['cards_sum'] = graphdata['player_card_1'] 
    for i in range(2,step+2):
        graphdata.loc[:,'cards_sum'] += graphdata.loc[:,f'player_card_{i}'] 
    hands_normalizer(graphdata,step+1)
    graphdata = graphdata.replace({ 1 : 'Win', 0 : 'Push' ,-1 : 'Loss'})
    return px.histogram(graphdata, x='cards_sum', color='win' , hover_data=['win'] , text_auto=True , labels={
                     'cards_sum': 'Player\'s Cards Sum', 'count' : 'Counts'} ,
                      color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"}, 
                      title=f'Step {step}')

@widgets.interact
def win_bar_graphs(step=widgets.IntSlider(value=1, min=1, max=6, step=1)):
    return win_graph_generator(data,step)

Not suprisingly , we can see that when the players sum is 21, regardless of the step, it either won or had a push (as it cant lose), and once the sum is above 22 it always lost therefore entirely red in our chart.

In [ ]:
def win_total_graph_generator(data,selection,norm):
    if selection == 'dealer':
        graphdata = data[['dealer_final_value','win']].replace({ -1 : 'Win', 0 : 'Push' ,1 : 'Loss'})  # Normalizing win for dealer
    elif selection == 'player':
        graphdata = data[['player_final_value','win']].replace({ 1 : 'Win' , 0 : 'Push' ,-1 : 'Loss'})
    if norm == False:
        return px.histogram(graphdata, x=f'{selection}_final_value', color='win' , hover_data=['win'] , labels={
                     f'{selection}_final_value': f'{selection}\'s Cards Sum', 'count' : 'Counts'} , text_auto=True ,
                      color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"}, 
                      title=(f'{selection}\'s distribution per sum of cards'))
    if norm == True:
        return px.histogram(graphdata, x=f'{selection}_final_value', color='win' , hover_data=['win'] , barnorm = 'percent' , labels={
                     f'{selection}_final_value': f'{selection}\'s Cards Sum', 'count' : 'Counts'} , text_auto=True ,
                      color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"}, 
                      title=(f'{selection}\'s distribution per sum of cards'))
    
@widgets.interact
def win_total_bar_graphs(selection= widgets.ToggleButtons( options=['player', 'dealer'] , value='player' , description='Selection') , norm = widgets.ToggleButtons( options=[True,False] , value=False , description='Normalized')):
    return win_total_graph_generator(data,selection,norm)

Keep in mind the win refers to the side youre viewing (i.e - win in dealer's graph means the dealer **won** and player **lost**)

We might notice a strange behavior when the dealer's sum of cards is over 21. The colors are partially green which means the dealer won with a sum of cards of above 21 and did not bust. If we recall, the dealer is the last to play, so if the player busted, the dealer wins regardless his sum of cards. We can also see the minimum value in the chart is 17 which makes sense as the dealer must play until it reaches a minimum sum of 17.

Simplified box plot of win per dealer/player final value.

In [ ]:
def win_box_plot_generator(data,side):
    if side == 'dealer':
        return px.box(data.replace({ 1 : 'Loss', 0 : 'Push' ,-1 : 'Win'}), x='win', y='dealer_final_value' , color='win' ,
                      color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"}, 
                      title=f'Win distribution')
    elif side == 'player':
        return px.box(data.replace({ 1 : 'Win', 0 : 'Push' ,-1 : 'Loss'}), x='win', y='player_final_value' , color='win' ,
                      color_discrete_map={ 'Loss': '#FC2020', 'Win': '#09D138' , 'Push': "#009999"}, 
                      title=f'Win distribution')

@widgets.interact
def show_win_box_plot(side = widgets.ToggleButtons( options=['player','dealer'] , value='player' , description='side')):
    return win_box_plot_generator(data,side)

Keep in mind the win refers to the side youre viewing (i.e - win in dealer's graph means the dealer **won** and player **lost**)

After the statistical extraction and comparasion to the theoretical statistics and probabilities, we can see our data corresponds to it. We can proceed to build our required models.

# Modeling

We will now create a model to try to predict results of a blackjack hands given the data we have.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

We first start with correlation matrix to see if there is any correlation in our data

In [ ]:
corrmatx = data.corr()
corrmatx

From a brief look we can see there are no correlating values, but since its a large dataset its better to visualize it.

In [ ]:
px.imshow(corrmatx)

Almost no correlation, except trivial pairs such as run_count-true_count or dealer_card_1 - dealer_up , which will be corrected. Also some of the pixels are N/A due to lack of information, will also be corrected.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import classification_report

Preprocessing the data before modeling:

In [ ]:
prepdata = data.sample(n=30000)  

We chose n=30000 due to computational time. In general you can choose whatever value you want.

We will now remove a lot of data from this table. The removed data is one of 2 categories:

-**Duplicate data**: Data that exists in 2 or more columns in any form

**-Non relevant data**: Data that is not relevant to the solution

In [ ]:
prepdata.drop(['dealer_up','initial_hand','dealer_final','dealer_final_value','player_final','player_final_value','actions_taken'],axis=1,inplace=True)  # Duplicate data
prepdata.drop(['shoe_id'],axis=1,inplace=True) # Not relevant

As said beforem the shoe_id is not relevant , neither are the actions_taken, 

Now we will try to get our wanted data prepared for modeling.

In [ ]:
prepdata.info()

We can see player and dealer cards columns contains NaNs. Since cards are categorial data, we can fill all NaNs with -1 for example. We can also see some columns are of float type and not int, which is not acceptable by the KNN. We will turn the values to integers for the KNN as all our values are singular

In [ ]:
prepdata[['action_taken_1','action_taken_2','action_taken_3','action_taken_4','action_taken_5','action_taken_6']] = prepdata[['action_taken_1','action_taken_2','action_taken_3','action_taken_4','action_taken_5','action_taken_6']].replace({ 'S' : 1, 'H' : 2 , 'D' : 3 , 'P' : 4 , 'R' : 5 , 'N' : 6})

In [ ]:
prepdata = prepdata.fillna(0).astype('int')
prepdata.info()

Now lets see our correlation matrix after all processing:

In [ ]:
px.imshow(prepdata.corr())

In [ ]:
prepdata['win'].value_counts()

The sample data is also (unsuprisingly) unbalanced in the same proportions as the mother-data. We must consider that in some models that can not handle unbalanced data.

And now our data is ready for modeling!

For the modeling, we will weigh mostly on percision for scoring, as we want the least false classifications (we dont want to donate any money to the casino..) 

## KNN

The first model we will start with is KNN , since its the simplest classification model and handles unbalanced data quite well. We will run this model for several hyper parameters (k) to look for the best performing model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNNlist = [x for x in range(1,26,2)]
acclist = []
perlist = []
conflist = {}
x_train,x_test,y_train,y_test= train_test_split(prepdata.drop('win',axis=1),prepdata['win'], test_size=0.2,random_state=42)
for k in KNNlist:
    knnfittype = KNeighborsClassifier(n_neighbors=k)
    knnfittype.fit(x_train,y_train)
    y_pred = knnfittype.predict(x_test)
    accuracy = metrics.accuracy_score(y_test,y_pred)
    percision =  metrics.precision_score(y_test,y_pred,average='weighted')
    confmatx = pd.DataFrame({'True':y_test, 'Predicted': y_pred}).groupby(['True','Predicted']).size()
    acclist.append(accuracy)
    perlist.append(percision)
    conflist[k] = confmatx
totalscore = pd.DataFrame({'Accuracy' : acclist , 'Percision' : perlist} , index = KNNlist)
totalscore

In [ ]:
def knn_confusin_matrix(conflist,step):
    return print(conflist.get(step))

@widgets.interact
def knn_confusin_matrix_widg(step=widgets.IntSlider(value=1, min=1, max=26, step=2 , description='K Value')):
    return knn_confusin_matrix(conflist,step)

In [ ]:
accperfig = make_subplots(rows=2, cols=1 , subplot_titles=("Accuracy vs k for KNN model", "Accuracy vs k for KNN model"))

# Add traces
accperfig.add_trace(go.Scatter(x=totalscore.reset_index()['index'], y= totalscore.reset_index()['Accuracy'],
                    mode='markers',
                    name='Accuracy') ,  row=1, col=1).update_traces(marker=dict(size=12,line=dict(width=1,color='DarkSlateGrey')))

accperfig.add_trace(go.Scatter(x=totalscore.reset_index()['index'], y= totalscore.reset_index()['Percision'],
                    mode='markers',
                    name='Percision') ,  row=2, col=1).update_traces(marker=dict(size=12,line=dict(width=1,color='DarkSlateGrey')))

accperfig.update_layout(height=800, width=900, title_text="Accuracy & Percision for KNN model")
accperfig.show()

We did not get good accuracy/percision, for any hyper parameter. Perhaps one of the reason is the very high dimension of the model.There are 24 columns to fit therefor the dimension is 24, which might make it difficult to fit the data. Another thing we can notice is that while the model is classifying the 'Win' and 'Loss' hands better as k is bigger, its struggling to fit the Push hands. 

k varies heavily on the randomly chosen data therefore we can not conclude what k is the best for our model (For each run we get different k value with the best accuracy and percision)

## Random Forest

The next method we'll choose is *Random Forest*. We'll choose this method as its simple and easy for discrete and low-ranged data like this. Unlike the KNN, it does not handle unbalanced data quite well, however scikit-learn has a function to tackle those kind of situations, so we'll use it when fitting.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(prepdata.drop('win',axis=1),prepdata['win'],test_size=0.2,random_state=42)
rffittype = RandomForestClassifier(class_weight= 'balanced')
rffittype.fit(x_train,y_train)
y_pred = rffittype.predict(x_test)
print(classification_report(y_test, y_pred , digits = 3))

In [ ]:
pd.DataFrame({'True':y_test, 'Predicted': y_pred}).groupby(['True','Predicted']).size()

We can see we got good percision relatively ( ~ 84%) and F1 score (~ 83%).As said its perhaps because our data is relatively easy to fit for decision trees.

We will also take advantage of this method to check for the importance of our parameters:

In [ ]:
importance = rffittype.feature_importances_
params = prepdata.drop('win',axis=1).columns
importancetable = pd.DataFrame({'Parameter' : params , 'Importance' : importance})
px.bar(importancetable, x='Parameter', y='Importance')

As expected, the parameters with the least occurences (cards and actions that were pulled after a lot of cards before) had the least importance contribution. However we can not ignore then as they are still a part of a hand and we will lose important info. Besides then almost all other params had more or less equal contribution.

## Gradient Boosting

We would also test the *Gradient Boosting* model, as its also a form of decision tree and usually outperforms the Random Forest method. It is usually used when there is concern of high bias (overfit) in regular decision tree, which *might* be our case (since our data is very simple and can fall into the overfit territory). The problem with this method that as it also handles unbalanced data quite bad, it does not have an easy and explicit solution like the Random Forest, so we will have to tackle this issue already in the test train split. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(prepdata.drop('win',axis=1),prepdata['win'],test_size=0.2,random_state=42)
gbfittype = GradientBoostingClassifier()
gbfittype.fit(x_train,y_train)
y_pred = gbfittype.predict(x_test)
print(classification_report(y_test, y_pred, digits = 3))

In [ ]:
pd.DataFrame({'True':y_test, 'Predicted': y_pred}).groupby(['True','Predicted']).size()

However, inspite of the good accuracy and percision relatively, we can see it did not perform Random Forest in our case! It mostly fails to classify the 'Push' situations.

**Logistic Regression**

Once last method we will try is *Logistic Regression*. It is intuitively the best method to chose as we are fitting Win-Loss situations, however the addition of Push situation makes it a bit more complex. Therefore we would fit it in the *One vs. Rest* method

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(prepdata.drop('win',axis=1),prepdata['win'],test_size=0.2,random_state=42)
logfittype = LogisticRegression(multi_class='ovr', solver='liblinear' , class_weight= 'balanced')
logfittype.fit(x_train,y_train)
y_pred = logfittype.predict(x_test)
print(classification_report(y_test, y_pred, digits = 3))
accuracy = metrics.accuracy_score(y_test,y_pred)
percision =  metrics.precision_score(y_test,y_pred,average='weighted')
pd.DataFrame({'True':y_test, 'Predicted': y_pred}).groupby(['True','Predicted']).size()

Unlike our intuition, the model does not perform quite well. Perhaps because the linear correlation between the parameters is not so strong. Once again we see there was a problem classifying the Push situations.

# Summary

In this notebook, we managed to develop a model with ~85% percision that predicts results of a blackjack hand. We also showed statistically proven situations where the odds can be tilted to the player's favor and what action should be taken at each step to increase the odds of winning.

The notebook can be further developed to a further statistical evaluation of the game, cards counting and model improving.